In [20]:
import geemap
import ee
from geemap import chart
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [21]:

ee.Initialize()
# ee.Authenticate()

##  Change Detection

In [22]:
choose_region=widgets.Dropdown(
    options=[ 'Thane', 'Mumbai_Suburban','Custom Region'],
    value='Custom Region',
    description='Select region ',
    disabled=False,
)
hbox1 = widgets.HBox([choose_region])

In [23]:
style = {"description_width":"initial"}
classify = widgets.Button(
    description='Compute change',
    button_style='primary',
    tooltip='Click to compute change in the selected region',
    style=style
)
classify.style.button_color='#11b981'

output=widgets.Output()

In [24]:


style = {"description_width":"initial"}

start_year = widgets.IntSlider(description='Start Year:', value=2018, min=2018, max=2020, style=style)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=1984, max=2020, style=style)



hbox3 = widgets.HBox([start_year])
hbox4 = widgets.HBox([end_year])

In [25]:
Map=geemap.Map()

In [26]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*") \
      .copyProperties(image, ["system:time_start"])


In [27]:

def calculate_change(t):
      global Map
      if choose_region.value=='Thane': 
        region=thane
      elif choose_region.value=='Mumbai Suburban':
        region=mumbai_suburban
      else:
        region=ee.FeatureCollection(Map.draw_features)
      
      admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
      mumbai_suburban=admin2.filter(ee.Filter.eq('ADM2_NAME', 'Mumbai Suburban'))
      thane = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Thane'))
      s2 = ee.ImageCollection("COPERNICUS/S2")
      

      start_date_1= str(start_year.value)+'-'+'01' + '-01'
      start_date_2 = str(start_year.value)+'-'+'12' + '-31'
      end_date_1 = str(end_year.value)+'-'+'01' + '-01'
      end_date_2 = str(end_year.value)+'-'+str(12) + '-30'
      with output:
        output.clear_output()
        print('Computing change.....')

        rgbVis = {
          'min': 0.0,
          'max': 3000,
          'bands': ['B4', 'B3', 'B2'],
        }
        filtered = s2 \
          .filter(ee.Filter.date(start_date_1,start_date_2)) \
          .filter(ee.Filter.bounds(region)) \
          .map(maskS2clouds)

        filtered2 = s2 \
          .filter(ee.Filter.date(end_date_1,end_date_2)) \
          .filter(ee.Filter.bounds(region)) \
          .map(maskS2clouds)

        image2019 = filtered.median().clip(region)
        image2020 = filtered2.median().clip(region)

  
        Map.addLayer(image2019, rgbVis, '2019')
        Map.addLayer(image2020, rgbVis, '2020')

        ndvi2019 = image2019.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
        ndvi2020 = image2020.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
        ndviVis = {'min':0, 'max':1, 'palette': ['white', 'green']}

        Map.addLayer(ndvi2019, ndviVis, 'ndvi')
        Map.addLayer(ndvi2020, ndviVis, 'ndvi')


        # 3. Classify NDVI into 5 classes
        ndvi2019classify = ee.Image(1) \
                  .where(ndvi2019.gt(0.0).And(ndvi2019.lte(0.2)), 2) \
                  .where(ndvi2019.gt(0.2).And(ndvi2019.lte(0.4)), 3) \
                  .where(ndvi2019.gt(0.4).And(ndvi2019.lte(0.6)), 4) \
                  .where(ndvi2019.gt(0.6), 5).clip(region)

        ndvi2020classify = ee.Image(1) \
                  .where(ndvi2020.gt(0.0).And(ndvi2020.lte(0.2)), 2) \
                  .where(ndvi2020.gt(0.2).And(ndvi2020.lte(0.4)), 3) \
                  .where(ndvi2020.gt(0.4).And(ndvi2020.lte(0.6)), 4) \
                  .where(ndvi2020.gt(0.6), 5).clip(region)

        # Add map layers
        Map.addLayer(ndvi2019classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI for start date',True)
        Map.addLayer(ndvi2020classify, {'min': 1, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI for end date',True)
        distance = ndvi2020.subtract(ndvi2019)
        Map.addLayer(distance, {'min': 0, 'max': 0.1, 'palette': ['white', 'green']}, 'Spectral distance')
        Map.centerObject(region, 9)
        print('Computation complete....')
classify.on_click(calculate_change)
 




 


In [28]:
hbox3

In [29]:
hbox4

In [30]:
hbox1

In [31]:
classify

Button(button_style='primary', description='Compute change', style=ButtonStyle(button_color='#11b981'), toolti…

In [32]:
output

Output()

In [33]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [34]:
# threshold=0.3

# changes = distance.lte(threshold).Or(distance.gte(threshold)).selfMask()
# # Calculating area above the threshold value
# pixel_area=changes.multiply(ee.Image.pixelArea())
# area=pixel_area.reduceRegion(reducer=ee.Reducer.sum(), geometry=thane, scale=30 ,maxPixels=10e9)

